데이터 불러오기

In [ ]:
import pandas as pd

products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/products.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/orders.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/order_products__train.csv')
departments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/departments.csv')
aisles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/aisles.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/order_products__prior.csv')

print('products shape :', products.shape)
print('orders shape :', orders.shape)
print('order_products_train shape :', order_products_train.shape)
print('departments shape :', departments.shape)
print('aisles shape :', aisles.shape)
print('order_products_prior shape :', order_products_prior.shape)

products shape : (49688, 4)
orders shape : (3421083, 7)
order_products_train shape : (1384617, 4)
departments shape : (21, 2)
aisles shape : (134, 2)
order_products_prior shape : (32434489, 4)


# Count 기반

Recommend_Product.ipynb 파일에서와 마찬가지로 유저특성의 정보훼손을 막고자 유저를 기준으로 데이터를 축소해줄 수 있는 함수를 만들어줍니다. 그리고 유저별 주문별 상품들의 리스트를 받는 것이 아니라 이번엔 유저가 주문번호 상관없이 주문한 모든 상품들을 리스트로 만들어줍니다. 즉, product_id의 중복을 허용하여 주문했던 상품들에 대한 기록이 모두 들어가 줄 수 있게 리스트로 만들어줍니다.

In [ ]:
def limit_data(num):
  arg = orders[(orders['user_id']<=num) & (orders['eval_set']=='prior')]
  arg.drop(['eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], axis=1, inplace=True)
  order_products_prior.drop(['add_to_cart_order'], axis=1, inplace=True)
  arg = arg.merge(order_products_prior, on='order_id')
  arg = arg.groupby(['user_id'])['product_id'].apply(list)
  arg = arg.reset_index()
  return arg

유저번호 1~1000에 해당하는 전체 주문된 상품 리스트

In [ ]:
data = limit_data(1000)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
data

user_id                                         product_id
0          1  [196, 14084, 12427, 26088, 26405, 196, 10258, ...
1          2  [32792, 47766, 20574, 12000, 48110, 22474, 165...
2          3  [9387, 17668, 15143, 16797, 39190, 47766, 2190...
3          4  [36606, 7350, 35469, 2707, 42329, 7160, 1200, ...
4          5  [15349, 21413, 48775, 28289, 8518, 11777, 3171...
..       ...                                                ...
995      996  [47193, 44830, 31506, 25050, 8186, 25530, 1295...
996      997  [45066, 24281, 10603, 141, 19446, 12206, 24852...
997      998  [44980, 13176, 7781, 1940, 49235, 22474, 27514...
998      999  [41844, 40001, 24852, 5212, 27966, 45763, 2294...
999     1000  [30492, 14870, 36865, 28465, 26165, 19660, 188...

[1000 rows x 2 columns]

유저별로 한번 주문시 평균적으로 장바구니에 몇개의 상품이 담기는지를 계산한 u_avg_basket칼럼을 만들어줍니다. 만드는 목적은 유저별로 평균적으로 몇개의 상품을 장바구니에 담는지 파악한 후 그 평균 개수만큼의 상품들을 추천해주기 위함입니다.

In [ ]:
prior = pd.merge(orders, order_products_prior, on = 'order_id', how = 'inner')
basket_amount = prior.groupby(['user_id', 'order_id'])['product_id'].count().to_frame('basket_amount').reset_index()
u_avg_basket = basket_amount.groupby('user_id')['basket_amount'].mean().to_frame('u_avg_basket').reset_index()
u_avg_basket.head()

user_id  u_avg_basket
0        1      5.900000
1        2     13.928571
2        3      7.333333
3        4      3.600000
4        5      9.250000

In [ ]:
u_avg_basket['u_avg_basket'] = u_avg_basket['u_avg_basket'].apply(round)

In [ ]:
u_avg_basket.head()

user_id  u_avg_basket
0        1             6
1        2            14
2        3             7
3        4             4
4        5             9

해당 유저가 특정 상품을 얼마나 구매했는지를 알 수 있도록 딕셔너리를 통해 데이터를 저장하고 횟수만큼 내림차순 정렬하여 상위 몇개만큼을 보여주는 리스트를 반환하는 함수를 정의합니다.
즉 위의 data테이블에서 구한 유저가 주문한 모든 상품리스트를 카운트하여 특정 상품이 몇개만큼 주문되었는지를 확인하고 그 중에서 가장 많이 주문된 상품들을 추천해는 것입니다.

ex) ['딸기', '딸기', '바나나', '초코파이'] => {'딸기':2, '바나나':1, '초코파이':1}

  딸기가 2개로 가장 많이 주문되었으니 유저는 딸기를 좋아하는게 확실하다!! 고로 딸기를 추천리스트로!! 대신 유저가 평균적으로 장바구니에 2개의 상품을 담으니깐 바나나까지 얹어서 딸기, 바나나를 최종 추천리스트로!!

In [ ]:
from collections import Counter

def user(df, user_id):
  bound = u_avg_basket[u_avg_basket['user_id']==user_id]['u_avg_basket'].values[0]
  a = Counter(df[df['user_id']==user_id]['product_id'].values[0]).most_common(bound)
  return [products[products['product_id']==k]['product_name'].values[0] for k, v in a]

user_id=10 유저의 추천 리스트(이 유저는 평균적으로 29개의 상품을 장바구니에 넣음, 그래서 그 만큼의 상품들을 추천)

In [ ]:
user(data, 3)

['Vanilla Unsweetened Almond Milk',
 'Organic Avocado',
 'Organic Baby Spinach',
 'Granny Smith Apples',
 'Unsweetened Chocolate Almond Breeze Almond Milk',
 'Organic Peeled Whole Baby Carrots',
 'Garlic Couscous']

In [ ]:
pd.DataFrame(user(data, 3), columns=['user_id=3, Recommend List'])

user_id=3, Recommend List
0                  Vanilla Unsweetened Almond Milk
1                                  Organic Avocado
2                             Organic Baby Spinach
3                              Granny Smith Apples
4  Unsweetened Chocolate Almond Breeze Almond Milk
5                Organic Peeled Whole Baby Carrots
6                                  Garlic Couscous